In [1]:
from flask import Flask, render_template, request, url_for, redirect, flash
import urllib.request
import os
from werkzeug.utils import secure_filename
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from gevent.pywsgi import WSGIServer


model = load_model('oldest surviving buildings in the world6.h5')

# to make dictionary from names:
names= ['Colosseum', 'Temple of Garni', 'Dun Carloway', 'Ruwanwelisaya', 'Great Wall of China', 'Mausoleum of the First Qin Emperor', 'Dhamek Stupa', 'Sanchi Stupa', 'Thracion Tomb of Kazanlak', "Naveta d'Es Tudons", 'Chogha Zanbil', 'Pyramids of Giza', 'Bent Pyramid', 'Pyramid of Djoser', 'Dolmens of North Caucasus', 'Ggantija']
lst= list(range(len(names)))

code= dict()
for key, value in zip(names, lst):
  code[key] = value
  code[key] = value
  code[key] = value
  code[key] = value

def getcode(n) : 
    for x , y in code.items() : 
        if n == y : 
            return x     


# to make dictionary from first_built:
first_built= ['70-80 A.D', 'C. 77 A.D', '100 B.C', '140 B.C', '221 B.C', 'Began Constructurction 240 B.C, Finished 208 B.C', '249 B.C, Rebuilt C. 500 A.D', '300 B.C', '300-400 B.C', '1200-750 B.C', '1250 B.C', '2550-2490 B.C', '2580 B.C', '2667-2648 B.C', '3000 B.C', '3700 B.C']

fst_built= dict()
for key, value in zip(first_built, lst):
  fst_built[key] = value
  fst_built[key] = value
  fst_built[key] = value
  fst_built[key] = value

def get_first_built(n) : 
    for x , y in fst_built.items() : 
        if n == y : 
            return x  

        
app= Flask(__name__)
app.secret_key= "nooormamdouh65"
UPLOAD_FOLDER= 'static/uploads/'
app.config['UPLOAD_FOLDER']= UPLOAD_FOLDER

ALLOWED_EXTENSIONS= set(['png', 'jpg', 'jpeg', 'gif'])
def allowed_file(filename):
    return filename.split('.')[1].lower() in ALLOWED_EXTENSIONS


@app.route('/')
def homepage():
    return render_template("index proj2.html")

@app.route('/', methods= ['POST'])    # دي معمول كده route زي ما ال methods= ['POST']  بتاعها لازم يكون '/' وال action اللي ال template اللي في ال form يعني الفانكشن دي هتشتغل مع ال POST أنا هنا هشتغل
def upload_image():
    if 'file' not in request.files:   # مش أسم تاني 'file' اللي أسمه type من نوع input فأنت كده في المكان الغلط او الفانكشن الغلط لأن الفانكشن دي بتشتغل علي التاج 'file' بتاعه مأسمهوش name ده لو ال file اللي نوعه input لو التاج ال    
        flash('No File Part')
        return redirect(request.url)

    file= request.files['file']       # request template اللي في ال name="file" الفايل او الصوره اللي هتتبعت هتبقي اللي جت من التاج اللي ال
    if file.filename == '':           # فلو الفايل اللي اتبعت ماليهوش اسم يعني لو اسمه فاضي فمعني كده ان مافيش جاجه اتبعتت يعني مافيش صوره اتبعتت فبعد ازنك مافيش صوره اترفعت لينا وارجع تاني للصفحه الرأيسيه ارفعلي صوره تانيه
        flash('No Image Selected For Uploading')
        return redirect(request.url)  # فأرجعلي عليالصفحه الرأيسيه هاتلي صوره تانيه
    
    if file and allowed_file(file.filename):         # ALLOWED_EXTENSIONS بتوع extensions بتاع الصوره من ضمن ال extension موجود فيه صور فعلاً وكمان اسم ال file تب لو ال
      
        filename= secure_filename(file.filename)     # بتاع الصوره secure_filename يبقي اسم الصوره هو ال 
        print(filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))     # ['static/uploads/أسم الصوره'] واحفظلي الصوره في المسار ده
        flash('Image Succuessfully Uploaded & Displayed Below')            # render_template وقتها بقي قولي أن الصوره فعلاً اترفعت وسيتم عرضها في الاسفل مباشراً وفعلا هروح اعرضها ب
    

        img_array= plt.imread(file)        # read image as array
        s= 150
        img_resize= cv2.resize(img_array, (s, s))
        img_gray= cv2.cvtColor(img_resize, cv2.COLOR_RGB2GRAY) 
        image_shape = (s, s, 1)
        img_gray = img_gray.reshape(1, *image_shape)
        # print(test_img.shape)
        test_img= img_gray / 255        # to normalize image
        y_pred = model.predict(test_img)
        # labels= ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
        pred_result= getcode(np.argmax(y_pred, axis=1))
        first_built_result= get_first_built(np.argmax(y_pred, axis=1))
        
        # country_code_lst= ['ita.svg', 'arm.svg', 'gbr.svg', 'sri.svg', 'chn.svg', 'chn.svg', 'ind.svg', 'ind.svg', 'bgr.svg', 'esp.svg', 'irn.svg', 'egy.svg', 'egy.svg', 'egy.svg', 'rus.svg', 'mlt.svg']   
        country_name_lst= ['Italy', 'Armenia', 'United Kingdom', 'Sir Lanka', 'China', 'China', 'India', 'India', 'Bulgaria', 'Spain', 'Iran', 'Egypt', 'Egypt', 'Egypt', 'Rusia', 'Malta']   
        arg_max= np.argmax(y_pred, axis=1)
        country_name= country_name_lst[arg_max[0]]
        
        return render_template('index proj2.html', filename= filename, pred_result= pred_result, first_built_result= first_built_result, country_name= country_name)
    
    else:
        flash('The uploaded image not in this types:  ["png", "jpg", "jpeg", "gif"]')       # ALLOWED_EXTENSIONS بتوع extensions موجود فيه صور فعلاً ولكن الصور مش من ضمن ال file تب لو ال
        return redirect(request.url)        # فأرجعلي تاني علي الصفحه الرأيسيه حملي صوره تانيه تكون من الأنواع المذكورة
    

# @app.route('/display/<filename>')
@app.route('/<filename>')
def display_image(filename):
    return redirect(url_for('static', filename= 'uploads/' + filename))


if __name__ == '__main__':
    app.run(port =3770)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:3770/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jun/2022 19:03:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2022 19:03:13] "GET /static/css/style%20pro2.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2022 19:03:16] "GET /static/js/main.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Jun/2022 19:03:17] "GET /static/js/jquery-3.6.0.min.js HTTP/1.1" 304 -
127.0.0.1 - - [22/Jun/2022 19:03:27] "GET /favicon.ico HTTP/1.1" 302 -
127.0.0.1 - - [22/Jun/2022 19:03:27] "GET /static/uploads/favicon.ico HTTP/1.1" 404 -


Colosseum5.jpg


127.0.0.1 - - [22/Jun/2022 19:06:28] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2022 19:06:28] "GET /Colosseum5.jpg HTTP/1.1" 302 -
127.0.0.1 - - [22/Jun/2022 19:06:28] "GET /static/uploads/Colosseum5.jpg HTTP/1.1" 200 -
